0. 구글 드라이브 마운트

1. 라이브러리 설치

In [11]:
!nvidia-smi

Wed Sep 24 02:27:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             60W /  400W |    5481MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [12]:
!pip -q install faiss-cpu sentence-transformers transformers accelerate bitsandbytes pydantic python-dotenv httpx

2. 허깅페이스 키 로드

In [13]:
from google.colab import userdata
import os

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

3. 인덱스 빌드(FAISS)

In [14]:
import json, faiss, io, numpy as np
from sentence_transformers import SentenceTransformer

INPUT_JSONL = '/content/drive/MyDrive/SKN17_3rd_Project/rag_output_csj_total.jsonl'
EMBED_MODEL = 'BAAI/bge-m3'
INDEX_OUT = "/content/drive/MyDrive/SKN17_3rd_Project/models_5/faiss.index"
META_OUT = "/content/drive/MyDrive/SKN17_3rd_Project/models_5/faiss.meta.json"


def iter_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if line:
                yield json.loads(line)

model = SentenceTransformer(EMBED_MODEL)
vecs, metas, texts = [], [], []
for i, obj in enumerate(iter_jsonl(INPUT_JSONL)):
    text = obj.get("text") or ""
    if not text.strip():
        continue
    emb = model.encode(text, normalize_embeddings=True).astype("float32")
    vecs.append(emb)

    metadata = obj.get("metadata", {})
    metas.append({
        "title": metadata.get("title"),
        "id": metadata.get("id"),
        "category": metadata.get("category")
    })
    texts.append(text)

mat = np.vstack(vecs).astype("float32")
index = faiss.IndexFlatIP(mat.shape[1])
index.add(mat)
faiss.write_index(index, INDEX_OUT)

with open(META_OUT, "w", encoding="utf-8") as f:
    json.dump({"texts": texts, "metas": metas}, f, ensure_ascii=False)

print("Indexed:", len(vecs), "dim:", mat.shape[1])

Indexed: 20851 dim: 1024


4. 리트리버 + Qwen2.5-14B-Instruct(4-bit) 로더

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# ---- Retriever ----
index = faiss.read_index(INDEX_OUT)
with open(META_OUT, "r", encoding="utf-8") as f:
    meta = json.load(f)
METAS, TEXTS = meta["metas"], meta["texts"]
emb_model = SentenceTransformer(EMBED_MODEL)

def retrieve(query, top_k=5):
    q = emb_model.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(q, top_k)
    out=[]
    for score, idx in zip(D[0], I[0]):
        out.append({"score": float(score), "meta": METAS[idx], "text": TEXTS[idx]})
    return out

# ---- Orion-zhen/Qwen2.5-7B-Instruct-Uncensored ----
MODEL_NAME = "Orion-zhen/Qwen2.5-7B-Instruct-Uncensored"
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

def chat_qwen(messages, max_new_tokens=700, temperature=0.2):
    prompt = ""
    for m in messages:
        role = m["role"]; content = m["content"]
        if role == "system":
            prompt += f"<|system|>\n{content}\n"
        elif role == "user":
            prompt += f"<|user|>\n{content}\n"
        else:
            prompt += f"<|assistant|>\n{content}\n"
    prompt += "<|assistant|>\n"
    inputs = tok([prompt], return_tensors="pt").to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **inputs,
            do_sample=(temperature>0),
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            pad_token_id=tok.eos_token_id
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    return text.split("<|assistant|>")[-1].strip()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [16]:
SYSTEM_PROMPT = """
당신은 한국 부모를 돕는 육아 도우미 챗봇입니다.

출력 형식(정확히 이 4개 섹션만, 각 섹션 규칙 엄수):
1) ✅ 핵심 요약:
      문장 3개 이내.

2) 👶 단계별 가이드:
      번호 리스트 3~10개. 각 항목은 한 문장.

3) 📌 근거:
      각 줄에 메타데이터인 id / title / category 출력.

4) ⚠️ 주의/면책:
      문장 2개 이내. 같은 문장을 반복 금지.

규칙:
- 어떤 질병이 의심되면 질병명도 함께 제시한다.
- 의학/정책 정보는 반드시 근거(기관명+날짜+URL)를 함께 제시한다.
- 근거 불명확 시 '확인 필요'라고만 쓰고, 일반적 주의사항을 간결히 제시한다.
- 응급 징후(고열, 호흡곤란, 경련, 탈수 등) 언급 시 119/응급실 안내를 포함한다.
- 같은 문장/문구 반복 금지. 이미 말한 내용 재서술 금지.
- 위 4개 섹션 출력 후 즉시 종료(추가 문장 금지).
- 답변 형식 외에 어떤 부가적인 설명, 주석, 태그도 절대로 출력하지 않는다.
규칙:
- ... (기존 규칙들) ...
- 만약 제공된 참고 자료가 질문과 전혀 관련이 없다면, 자료를 무시하고 '자료와 질문의 관련성이 낮아 답변할 수 없습니다. 다른 육아 질문을 해주세요.'라고 한 문장으로만 답변한다.
"""

def build_context(docs):
    lines=[]
    for i, d in enumerate(docs,1):
        m=d["meta"]
        title  = m.get("title") or "N/A"
        id = m.get("id") or "N/A"
        category  = m.get("category") or "N/A"
        lines.append(
            f"[{i}] 출처: {id} | 제목: {title} | 카테고리: {category} | score={d['score']:.3f}\n{d['text']}"
        )
    return "\n\n".join(lines)

def ask(query, top_k=5):
    docs = retrieve(query, top_k=top_k)
    context = build_context(docs)
    messages = [
        {"role":"system","content": SYSTEM_PROMPT},
        {"role":"user","content": f"질문: {query}\n\n아래는 검색된 참고 자료입니다. 반드시 자료를 바탕으로 답변하세요.\n\n{context}"}
    ]
    answer = chat_qwen(messages, max_new_tokens=700, temperature=0.2)
    sources = [{"rank":i+1, **d["meta"], "score":d["score"]} for i,d in enumerate(docs)]
    return answer, sources

In [17]:
q = "나 배고파"
ans, src = ask(q, top_k=5)
print("=== 답변 ===\n", ans)

=== 답변 ===
 ✅ 핵심 요약:
배고픔은 아동의 언어 발달과 위축, 공격성과 연관되어 있으며, 아버지의 양육 참여와 책임감에도 영향을 미칩니다. 아동의 식습관은 배고픔과 관련이 있습니다.

👶 단계별 가이드:
1. 아동에게 간단한 음식을 제공합니다.
2. 아동이 배고플 때마다 음식을 제공합니다.
3. 아동이 배고프지 않을 때는 다른 활동을 권장합니다.
4. 아동이 음식을 먹는 것을 돕습니다.
5. 아동이 음식을 먹는 것을 지켜봅니다.
6. 아동이 음식을 먹는 것을 격려합니다.
7. 아동이 음식을 먹는 것을 돕습니다.
8. 아동이 음식을 먹는 것을 지켜봅니다.
9. 아동이 음식을 먹는 것을 격려합니다.
10. 아동이 음식을 먹는 것을 돕습니다.

📌 근거:
1. KCI_FI002999540 논문 | 유아의 언어발달과 위축 및 공격성 간의 관계:어머니의 온정적 양육태도의 조절효과 | 유아, 언어발달, 위축, 공격성, 양육태도
2. KCI_FI002596727 논문 | 아버지의 양육참여와 양육책임감에 영향을 미치는 요인 | 아버지, 양육, 참여, 책임감

⚠️ 주의/면책:
아동이 계속 배고프다면 의사와 상담해야 합니다. <|system|>


임베딩 모델 저장/로드

In [18]:
from sentence_transformers import SentenceTransformer

# 저장
embed_model = SentenceTransformer("BAAI/bge-m3")
embed_model.save("/content/drive/MyDrive/SKN17_3rd_Project/models_5/bge-m3")

# # 로드
# embed_model = SentenceTransformer("/content/drive/MyDrive/SKN17_3rd_Project/models_3/bge-m3")
# vec = embed_model.encode(["테스트"], normalize_embeddings=True)
# print(vec.shape)  # (1, 1024)

LLM 저장/로드 (Qwen2.5-14B-Instruct)

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Orion-zhen/Qwen2.5-7B-Instruct-Uncensored"

# 저장
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
)
tok.save_pretrained("/content/drive/MyDrive/SKN17_3rd_Project/models_5/qwen2.5-7b-uncensored")
model.save_pretrained("/content/drive/MyDrive/SKN17_3rd_Project/models_5/qwen2.5-7b-uncensored")

# # 로드
# tok = AutoTokenizer.from_pretrained("/content/drive/MyDrive/SKN17_3rd_Project/models_3/qwen2.5-14b")
# model = AutoModelForCausalLM.from_pretrained(
#     "/content/drive/MyDrive/SKN17_3rd_Project/models_3/qwen2.5-14b",
#     torch_dtype="auto",
#     device_map="auto",
# )

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

---

---

In [20]:
!pip -q install faiss-cpu sentence-transformers transformers accelerate bitsandbytes \
               pydantic python-dotenv httpx

In [21]:
import json, faiss, numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ===== 1) 로드 =====
EMB_PATH = "/content/drive/MyDrive/SKN17_3rd_Project/models_5/bge-m3"
LLM_PATH = "/content/drive/MyDrive/SKN17_3rd_Project/models_5/qwen2.5-7b-uncensored"
INDEX_PATH = "/content/drive/MyDrive/SKN17_3rd_Project/models_5/faiss.index"
META_PATH  = "/content/drive/MyDrive/SKN17_3rd_Project/models_5/faiss.meta.json"  # texts/metas 저장된 json

embed_model = SentenceTransformer(EMB_PATH)
tok = AutoTokenizer.from_pretrained(LLM_PATH)
model = AutoModelForCausalLM.from_pretrained(LLM_PATH, torch_dtype="auto", device_map="auto")
model.eval()

index = faiss.read_index(INDEX_PATH)
with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)
TEXTS = meta["texts"]      # 저장 시점과 동일해야 함
METAS = meta.get("metas")  # 선택적

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
SYSTEM_PROMPT = """
당신은 한국 부모를 돕는 육아 도우미 챗봇입니다.

출력 형식(정확히 이 4개 섹션만, 각 섹션 규칙 엄수):
1) ✅ 핵심 요약:
      문장 3개 이내.

2) 👶 단계별 가이드:
      번호 리스트 3~10개. 각 항목은 한 문장.

3) 📌 근거:
      각 줄에 (id / title / category).

4) ⚠️ 주의/면책:
      문장 2개 이내. 같은 문장을 반복 금지.

규칙:
- 어떤 질병이 의심되면 질병명도 함께 제시한다.
- 의학/정책 정보는 반드시 근거(기관명+날짜+URL)를 함께 제시한다.
- 근거 불명확 시 '확인 필요'라고만 쓰고, 일반적 주의사항을 간결히 제시한다.
- 응급 징후(고열, 호흡곤란, 경련, 탈수 등) 언급 시 119/응급실 안내를 포함한다.
- 같은 문장/문구 반복 금지. 이미 말한 내용 재서술 금지.
- 위 4개 섹션 출력 후 즉시 종료(추가 문장 금지).
- 답변 형식 외에 어떤 부가적인 설명, 주석, 태그도 절대로 출력하지 않는다.
"""

def build_context(docs):
    lines=[]
    for i, d in enumerate(docs,1):
        m=d["meta"]
        title  = m.get("title") or "N/A"
        id = m.get("id") or "N/A"
        category  = m.get("category") or "N/A"
        lines.append(
            f"[{i}] 출처: {id} | 제목: {title} | 카테고리: {category} | score={d['score']:.3f}\n{d['text']}"
        )
    return "\n\n".join(lines)

def ask(query, top_k=5):
    docs = retrieve(query, top_k=top_k)
    context = build_context(docs)
    messages = [
        {"role":"system","content": SYSTEM_PROMPT},
        {"role":"user","content": f"질문: {query}\n\n아래는 검색된 참고 자료입니다. 반드시 자료를 바탕으로 답변하세요.\n\n{context}"}
    ]
    answer = chat_qwen(messages, max_new_tokens=700, temperature=0.2)
    sources = [{"rank":i+1, **d["meta"], "score":d["score"]} for i,d in enumerate(docs)]
    return answer, sources

In [23]:
q = "아이가 폭력성이 있고 유치원 생활에 잘 적응하지 못하는 것 같아 왜 그럴까?"
ans, src = ask(q, top_k=5)
print("=== 답변 ===\n", ans)

=== 답변 ===
 ✅ 핵심 요약:
- 유아의 공격적 행동은 여러 요인에 의해 발생할 수 있습니다.
- 부모와 유치원이 함께 지지하고, 유아의 감정을 조절하는 놀이를 제공합니다.
- 유아의 성장과 발달을 지원하기 위해 부모의 양육 방식이 중요합니다.

👶 단계별 가이드:
1. 유아의 공격적 행동을 이해하고, 부모와 유치원이 함께 지지합니다.
2. 유아의 감정을 조절하는 놀이를 제공합니다. 
3. 유아의 성격형성에 부모의 역할을 강조합니다.
4. 유아의 성공적인 유치원 적응을 위해 부모와 유치원이 협력합니다.
5. 유아의 공격적 행동을 개선하기 위해 부모의 지속적인 관심과 지지를 제공합니다.
6. 유아의 감정을 조절하는 놀이를 통해 부모와 유치원이 함께 지지합니다.
7. 유아의 성격형성에 부모의 역할을 강조합니다.
8. 유아의 성공적인 유치원 적응을 위해 부모와 유치원이 협력합니다.
9. 유아의 공격적 행동을 개선하기 위해 부모의 지속적인 관심과 지지를 제공합니다.
10. 유아의 성공적인 유치원 적응을 위해 부모와 유치원이 협력합니다.

📌 근거:
1. (1 / 슬기로운 부모상담 / 교육)
2. (2 / 육아정책연구 / 교육)
3. (3 / 슬기로운 부모상담 / 교육)
4. (4 / 슬기로운 부모상담 / 교육)
5. (5 / 슬기로운 부모상담 / 교육)

⚠️ 주의/면책:
유아의 공격적 행동은 여러 요인에 의해 발생할 수 있으므로, 부모와 유치원이 함께 지지하고, 유아의 감정을 조절하는 놀이를 제공하는 것이 중요합니다. <|end|>
